In [1]:
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader
#from gpt4all import GPT4All
from langchain.llms import GPT4All
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings
from langchain.chains import  LLMChain
from langchain.prompts import PromptTemplate
# from langchain.agents import TextGenerator
#from langchain.llms import TextGen


from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.callbacks.base import BaseCallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import numpy as np
import pandas as pd
from langchain_community.document_loaders import DataFrameLoader


In [2]:
def cosine_similarity(vec1, vec2):
    # Compute the dot product of vec1 and vec2
    dot_product = np.dot(vec1, vec2)
    
    # Compute the L2 norms (or magnitudes) of vec1 and vec2
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    
    # Compute the cosine similarity
    cos_sim = dot_product / (norm_vec1 * norm_vec2)
    
    return cos_sim

### Load data

In [39]:
#"youtube_transcript_whitney.pdf"
# estimating_demand_with_Machine_learning # interesting_speech
filename="youtube_ginger_benefits"
loader = PyPDFLoader(f"""data/{filename}.pdf""")
data = loader.load_and_split()

In [4]:
data[0].page_content

"#\ntactiq.io\nfree\nyoutube\ntranscript\n#\n'Cut\nThrough\nThe\nPolitical\nBS':\nVivek\nRamaswamy\nSpeaks\nTo\nCalifornia\nRepublican\nParty\nIn\nAnaheim,\nCA\n#\nhttps://www.youtube.com/watch/r8sUxBSHKJI\n00:00:00.359\nin\nthe\nthank\nyou\nit's\ngood\nto\nbe\nhere\n00:00:01.839\neverybody\nI\n00:00:04.160\nuh\nI'm\ngoing\nto\nget\nright\ninto\n00:00:07.000\nit\nbecause\nI\ndon't\nthink\nwe\nhave\na\nlot\n00:00:09.240\nof\ntime\nI\ndon't\nmean\ntime\ntoday\nin\nthe\n00:00:12.559\nschedule\nthough\nthat\nmay\nbe\ntrue\n00:00:15.440\ntoo\nI\ndon't\nthink\nwe're\nworking\nwith\na\n00:00:17.600\nlot\nof\ntime\nas\na\ncountry\nright\nnow\nI\n00:00:22.279\ncame\nhere\nI\nwas\nwith\nmy\ntwo\nsons\nthis\n00:00:23.720\nmorning\none\nis\nthree\nyears\nold\none\nis\n00:00:25.720\nOney\nold\nif\nthey\ngraduate\nfrom\nhigh\n00:00:28.359\nschool\nbefore\nwe've\ngotten\nthis\nright\nI\n00:00:30.160\ndon't\nthink\nwe\nhave\na\ncountry\nleft\nand\n00:00:32.279\nso\nI'm\ngoing\nto\nget\nright\ninto\nit

## Load the LLM Model

In [4]:
# get the model from https://gpt4all.io/index.html #too huge to keep in repo
local_path = (
    ".model/gpt4all-falcon-q4_0.gguf"  # replace with your desired local file path
)
#model = GPT4All("mistral-7b-instruct-v0.1.Q4_0.gguf")

# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
model = GPT4All(model=local_path, callbacks=callbacks, verbose=True)


In [40]:
# Constants: Define constants used in the code.
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 100 # control for chunk overlap by denoting how many characters of the previous chunk comes into the next chunk (for continuity)

# Create an instance of RecursiveCharacterTextSplitter with the desired chunk size and overlap
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP, 
    length_function=len  # function used to measure chunk size
)

# split documents into smaller chunks
splits = r_splitter.split_documents(data)


### Create embeddings 

In [41]:

# Use embeddings to combine the long form text into manageable token size
gpt4all_embd = GPT4AllEmbeddings()
vectorstore = Chroma.from_documents(documents=splits, embedding=GPT4AllEmbeddings())


bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522
bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [42]:
persist_directory=f"""chroma/sds/{filename}"""
vectorstore = Chroma.from_documents(documents=splits, embedding=GPT4AllEmbeddings(),persist_directory=persist_directory)
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=gpt4all_embd)

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


### Ask your question

In [43]:
# Additional context about the file
inital_context="The file contains the youtube transcript of a video."

In [86]:
question = "In the context of the video, Does eating ginger stop rainfall"
### what are the negative impacts of ginger, Does eating ginger prevent world war 3  #Can I consume ginger everyday
### "Are people of color more affected by home buying due to inflation? How does the video talk about this topic?"
### Summarize in 100 words on what is this video about
#### "Explain promo dampening to a 10 year old Kid"
#### Does the speaker sound like he wants the people to infiltrate

### Find embeddings of the question

In [87]:

# Find the top 5 documents relevant to the question
docs = vectorstore.similarity_search(question, k=10)

q_emb = gpt4all_embd.embed_query(question)
q_vec = np.array(q_emb)

In [88]:
for d in docs:
    emb = gpt4all_embd.embed_query(d.page_content)
    vec = np.array(emb)
    cosine = cosine_similarity(q_vec, vec)
    print(cosine)

0.46848413237012815
0.4677397978387059
0.4631087393687268
0.44613476899863985
0.4325937485255695
0.4054487087187896
0.4035181216286784
0.4032999918136225
0.40096226711634103
0.3970150987834641


In [89]:
for i in docs:
    print(i)

page_content="00:11:01.680\nthat\nGinger\nis\ngood\nfor\nand\nif\nyou\nadd\n00:11:04.260\nthis\nseasonal\ntreat\nto\nyour\ndaily\ndiet\n00:11:06.000\nyou\nmay\nfind\nthat\nit\nrelieves\na\nnumber\n00:11:07.920\nof\nsymptoms\nand\ndiscomfort\nby\ncovering\n00:11:10.440\nall\nthe\nbases\nGinger\nis\na\nversatile\n00:11:12.420\nremedy\nthat\nbelongs\nin\nevery\nhealth\n00:11:14.220\nconscious\nperson's\nkitchen\nmore\nthan\n00:11:16.740\nthat\nit\nhas\nanti-inflammatory\nproperties\n00:11:18.600\nso\nit's\nan\nexcellent\naddition\nto\ndieters\n00:11:20.940\ntoolkits\nenjoyed\nthis\nvideo\nhit\nthe\nlike\n00:11:24.240\nbutton\nto\nsubscribe\nto\nour\nchannel\nfor\n00:11:26.579\nmore\ninformative\nvideos\nlike\nthis\none\n00:11:28.680\nthanks\nfor\nwatching\nyour\nWellness\nis\nour\n00:11:32.040\nmission\n00:11:34.260\nforeign\n00:11:35.480\n[Music]" metadata={'page': 6, 'source': 'youtube_ginger_benefits.pdf'}
page_content='prevention\nresearch\nscientists\nexamined\n00:05:03.120\nthe\neff

In [90]:

# ### Create the Simple template for question and answer
# template = """
# Question: {question}
# Answer: Let's think step by step.
# """

# prompt = PromptTemplate(template=template, input_variables=["question"])

# llm_chain = LLMChain(prompt=prompt, llm=model)


In [91]:
# Define a prompt template. This is a format for the text input we'll give to our model.
# It tells the model how to structure its response and what to do in different situations.
template = """I will provide you pieces of [Context] to answer the [Question]. Provide answer based on the content and context provided. \
If you don't know the answer based on [Context] just say that you don't know, don't try to make up an answer. \
[Context]: {context} \
[Question]: {question} \
Helpful Answer:"""

final_template=inital_context + template

# If your answer includes any sort of list, return it in bullets. \
# Format your answer to Markdown. \

# Create a PromptTemplate object from our string template.
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)



# Create a RetrievalQA object. This uses our language model (llm) and a retriever,
# which is our vector database (vectordb). This object will handle asking our model questions
# and retrieving relevant documents to help answer them.
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=vectorstore.as_retriever(),
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)


# Ask our question to the qa_chain, and store the result.
result = qa_chain({"query": question})



 No, there is no evidence to suggest that eating ginger can prevent rainfall. The video discusses the potential health benefits of consuming ginger, including its anti-inflammatory and anti-nausea properties. However, it does not mention any connection between ginger consumption and rainfall.

In [92]:
from IPython.display import Markdown, display

def print_better_text(text):
    display(Markdown(text))

# Example usage
print_better_text("### Question: "+question+"\n###"+result["result"])

### Question: In the context of the video, Does eating ginger stop rainfall
### No, there is no evidence to suggest that eating ginger can prevent rainfall. The video discusses the potential health benefits of consuming ginger, including its anti-inflammatory and anti-nausea properties. However, it does not mention any connection between ginger consumption and rainfall.

In [29]:
for i in result['source_documents']:
    print(str(i).split('\\n'))

['page_content="vacancy', 'rate”', '00:03:17.530', 'over', 'about', '70', 'years.', '00:03:18.531', "It's", 'the', 'percent', 'of', 'homes', 'in', 'the', 'US', '00:03:20.533', 'that', 'are', 'actually', 'for', 'sale.', '00:03:22.076', 'And', "it's", 'currently', 'the', 'lowest', '00:03:23.703', 'since', 'we', 'started', 'collecting', 'data.', '00:03:25.955', 'And', 'that', 'is', 'partly', 'the', 'result', 'of', 'restrictive', '“zoning', 'laws”:', '00:03:30.168', 'locally-set', 'rules,', 'all', 'across', 'the', 'US,', '00:03:32.295', 'that', 'regulate', 'thingslike', 'where', 'homes', 'can', 'and', "can't", 'be', 'built,', '00:03:36.299', 'how', 'many', 'units', 'a', 'building', 'can', 'have', 'in', 'it," metadata={\'page\': 1, \'source\': \'median_price_vox.pdf\'}']
['page_content="00:07:18.688', 'the', '3%,', '4%,', '5%', 'zone', '--', '00:07:20.565', 'it', 'means', 'that,', 'since', 'selling', 'their', 'home', 'might', 'mean', '00:07:22.942', 'having', 'to', 'buy', 'a', 'new', 'one',